#Scrapping tweets from twitter
Using snscrape for scrapping twitter

In [ ]:
!pip install snscrape
!pip install wordcloud
!pip install transformers

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

# Created a list to append all tweet attributes(data)
tweets = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet_1 in enumerate(sntwitter.TwitterSearchScraper('Microsoft since:2022-11-03 until:2022-11-04').get_items()):
    if i>100:
        break
    tweets.append(tweet_1.content)
    
# Creating a dataframe from the tweets list above 
# tweets_df = pd.DataFrame(attributes_container, columns=["Date Created", "Number of Likes", "Source of Tweet", "Tweets"])
tweets_df = pd.DataFrame (tweets, columns= ['Tweets'])


In [ ]:
#function to clean the text
import re
import matplotlib.pyplot as pl
from textblob import TextBlob
from wordcloud import WordCloud
import numpy as np

def clean_text(text):
    text = re.sub(r'@[A-Za-z0-9]+','',text) #removed @ mentions
    text = re.sub(r'#', '', text) # removed # tags
    text = re.sub(r'RT[\s]+','', text) # removed Retweets
    text = re.sub(r'https?:\/\/\S+','', text) # removed hyperlinks

    return text

tweets_df['Tweets'] = tweets_df['Tweets'].apply(clean_text)
  

In [ ]:
#load the model and tokenizer
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from scipy.special import softmax

roberta = "cardiffnlp/twitter-roberta-base-sentiment"
model = AutoModelForSequenceClassification.from_pretrained (roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)



def getScores (text):
  encoded_tweet = tokenizer ( text, return_tensors = 'pt')
  output = model(**encoded_tweet)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  return scores



tweets_df['Sentiment'] = tweets_df['Tweets'].apply(getScores)

neg_t = 0
pos_t = 0
net_t = 0
for i in range(100):
  temp = tweets_df['Sentiment'][i][0]
  neg_t = neg_t + temp
  temp = tweets_df['Sentiment'][i][1]
  net_t = net_t + temp
  temp = tweets_df['Sentiment'][i][2]
  pos_t = pos_t + temp
avg_neg = neg_t/100
avg_net = net_t/100
avg_pos = pos_t/100
print('avg Positive = ', avg_pos)
print('avg Negative = ', avg_neg)
print('avg Neutral = ', avg_net)

In [ ]:

#sentiment analysis
# for i in range(100):
#   encoded_tweet = tokenizer ( tweets_df['Tweets'][i], return_tensors = 'pt')
# print(encoded_tweet)
  # output = model(**encoded_tweet)
  # scores = output[0][0].detach().numpy()
  # scores = softmax(scores)
# print(scores)
  # for i in range(len(scores)):
  #   l = labels[i] 
  #   s = scores[i]

  #   print(l,s)
# 1 negative 2 neutral 3 positive
# labels = ['Negative','Neutral', 'Positive']

In [ ]:
#using Polarity and subjectivity to analyse the sentiment
# create a function t get subjectivity

def getSubjectivity(text):
    return [TextBlob(text).sentiment.subjectivity , TextBlob(text).sentiment.polarity]

# Create two new column
# tweets_df['subjectivity'] = tweets_df['Tweets'].apply(getSubjectivity)
# tweets_df['polarity'] = tweets_df['Tweets'].apply(getPolarity)
tweets_df['subjectivity']  = tweets_df['Tweets'].apply(getSubjectivity)

tweets_df['subjectivity'][9][0]

In [ ]:
#make a wordcloud for the words apperiang in the tweets
from wordcloud import WordCloud
import matplotlib.pyplot as plt
allWords = ' '.join([twts for twts in tweets_df['Tweets']])
wc = WordCloud(width = 500, height = 300,  random_state = 21, max_font_size = 110).generate(allWords)

plt.imshow(wc, interpolation = "bilinear")
plt.axis('off')
plt.show()

In [ ]:
#create a function to compute negative neural and positive analysis
def getAnalysis (score):
  if score<0:
    return 'Negative'
  elif score ==0:
    return 'Neutral'
  else:
    return 'Positive'

tweets_df['Analysis']= tweets_df['polarity'].apply(getAnalysis)

tweets_df

